# MILP for COPP

In [40]:
# import modules
from gurobipy import *
import numpy as np
import pandas as pd
import pyodbc

In [41]:
# connect to the database
conn_str = 'DSN=JDATA;database=WarehouseModel;Trusted_Connection=yes;'
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

In [42]:
# get data from database
lineItem = pd.read_sql("SELECT ItemID FROM LineItem", conn)
pickers = pd.read_sql("SELECT PickerID FROM Pickers", conn)
transporters = pd.read_sql("SELECT TransporterID FROM Transporters", conn)

In [43]:
# define the set of all items, pickers and transporters. Decision variable a[e1,e2,i]
Item_P = pd.merge(lineItem, pickers, how='cross')
Item_P_T = pd.merge(Item_P , transporters, how='cross')
Item_P_T['Cell'] = 1
assignments, _ = multidict(Item_P_T.set_index(['PickerID', 'TransporterID', 'ItemID']).to_dict()['Cell'])
# define the set of all pickers and item picking sequence segments. Decision variabel X[e1,i,j]
Item_Item = pd.merge(lineItem, lineItem, how='cross')
Item_Item['Keep']=Item_Item.apply(lambda x: False if x['ItemID_x'] == x['ItemID_y'] else True, axis = 1)
Item_Item = Item_Item[Item_Item['Keep']]
P_Item_Item = pd.merge(pickers,Item_Item, how='cross')
Picking_segments, _ = multidict(P_Item_Item.set_index(['PickerID', 'ItemID_x', 'ItemID_y']).to_dict()['Keep'])
# define the set of all pickers and travel segements. 

In [32]:
Item_Item = pd.merge(lineItem, lineItem, how='cross')

In [35]:
Item_Item['Keep']=Item_Item.apply(lambda x: False if x['ItemID_x'] == x['ItemID_y'] else True, axis = 1)

In [44]:
Picking_segments

<gurobi.tuplelist (5940 tuples, 3 values each):
 ( 1 , 1  , 2  )
 ( 1 , 1  , 3  )
 ( 1 , 1  , 4  )
 ( 1 , 1  , 5  )
 ( 1 , 1  , 6  )
 ( 1 , 1  , 7  )
 ( 1 , 1  , 8  )
 ( 1 , 1  , 9  )
 ( 1 , 1  , 10 )
 ( 1 , 1  , 11 )
 ( 1 , 1  , 12 )
 ( 1 , 1  , 13 )
 ( 1 , 1  , 14 )
 ( 1 , 1  , 15 )
 ( 1 , 1  , 16 )
 ( 1 , 1  , 17 )
 ( 1 , 1  , 18 )
 ( 1 , 1  , 19 )
 ( 1 , 1  , 20 )
 ( 1 , 1  , 21 )
 ( 1 , 1  , 22 )
 ( 1 , 1  , 23 )
 ( 1 , 1  , 24 )
 ( 1 , 1  , 25 )
 ( 1 , 1  , 26 )
 ( 1 , 1  , 27 )
 ( 1 , 1  , 28 )
 ( 1 , 1  , 29 )
 ( 1 , 1  , 30 )
 ( 1 , 1  , 31 )
 ( 1 , 1  , 32 )
 ( 1 , 1  , 33 )
 ( 1 , 1  , 34 )
 ( 1 , 1  , 35 )
 ( 1 , 1  , 36 )
 ( 1 , 1  , 37 )
 ( 1 , 1  , 38 )
 ( 1 , 1  , 39 )
 ( 1 , 1  , 40 )
 ( 1 , 1  , 41 )
 ( 1 , 1  , 42 )
 ( 1 , 1  , 43 )
 ( 1 , 1  , 44 )
 ( 1 , 1  , 45 )
 ( 1 , 1  , 46 )
 ( 1 , 1  , 47 )
 ( 1 , 1  , 48 )
 ( 1 , 1  , 49 )
 ( 1 , 1  , 50 )
 ( 1 , 1  , 51 )
 ( 1 , 1  , 52 )
 ( 1 , 1  , 53 )
 ( 1 , 1  , 54 )
 ( 1 , 1  , 55 )
 ( 1 , 2  , 1  )
 ( 1 , 2  , 3  )
